# Receive and read streaming events

Here you want to create a process that will listen to a topic, and print all that it get from this topic

In [ ]:
# import dependancies : request to do some get or post queries directly in python
import requests

### 1.2.0 Consumer configuration

In [ ]:
kafkabrokers="https://streaming-bridge.course.aiengineer.codex-platform.com"
# as every personal object, consumer is name with name-surname
consumerName=''#name-surname
keyName=''#name-surname

In [ ]:
#header when create and suscribe with consumer group
headers={
    'content-type': 'application/vnd.kafka.v2+json'
}
#header when reading from a topic 
headersGet={
   'Accept': 'application/vnd.kafka.binary.v2+json'
}
# topic dict definition
topic = {
    "topics": [
        "aiengineer.tp.1"
    ]
}
# consumer configuration
consumerConf = {
    "name": f"{consumerName}",
    "auto.offset.reset": "earliest", # when reseting offset (index of last red message) the chosen index is the earliest
    "format": "binary", #messages are binary encoded
    "enable.auto.commit": False, # we must manually say to the kafka bridge what are the consumed messages offset
    "fetch.min.bytes": 512,
    "consumer.request.timeout.ms": 30000
}


### 1.2.1 Create your consumer group

In [ ]:
# this will allow you to suscribe to a topic
resp = requests.post(url = f"{kafkabrokers}/consumers/consumer-group-{consumerName}",
                  headers=headers,
                  json=consumerConf
                  )
print(resp.status_code, resp.text)


### 1.2.2 Suscrible to a topic with your consumer, part of the consumer group

In [ ]:
requests.post(url = f"{kafkabrokers}/consumers/consumer-group-{consumerName}/instances/{consumerName}/subscription",
                  headers=headers,
                  json=topic
                  ).content


### (1.2.3) Delete consumer group

In [ ]:
# Not needed but in case of wrong consumer group creation, you can delete it from here
# resp = requests.delete(url=f"{kafkabrokers}/consumers/consumer-group-{consumerName}/instances/{consumerName}")


### 1.2.4 Get events from the topic

In [ ]:
# import dependancies : a python client that allow writing events to our kafka bridge
from kafka_bridge_client import KafkaBridgeConsumer
# import depdendancies : a library tha allow easy encoding/decoding management
import codecs

In [ ]:
topic.get('topics').pop()

In [ ]:
consumer1 = KafkaBridgeConsumer(
    topic.get('topics').pop(),
    group_id=f"consumer-group-{consumerName}",
    auto_offset_reset='earliest',
    enable_auto_commit=False,
    bootstrap_server=kafkabrokers,
    consumer_name=consumerName
)

In [ ]:
def decode_kafka_record(record):
    """
    in: record : kafka_bridge_client.Message
    return : key string, value string
    """
    key = codecs.decode(bytes(record['key'],'utf-8'),'base64').decode('utf-8')
    value = codecs.decode(bytes(record['value'],'utf-8'),'base64').decode('utf-8')
    return key, value
    

In [ ]:
#Here we will asynchronously get records from topics
async for rec in consumer1.get_records(max_bytes=10000):
    ## get decoded key and value from record
    key,value = decode_kafka_record(rec)
    ## print those corresponding to your user key
    if key==keyName:
        print(f"record key: {key}")
        print(f"record valaue: {value}")
    await consumer1.commit()

### There shoul be printed your name as a key and a line of the features as values

#### Now stop the kernel of your notebook (using the stop icon) to terminate the topic reading